In [ ]:
import numpy as np


def detrend_segment(segment):
    ### detrending the each segment using linear regression
    x = np.arange(len(segment))
    slope, intercept = np.polyfit(x, segment, 1)
    detrended_segment = segment - (slope * x + intercept)
    return detrended_segment

def dfa_exponent(signal_profile, segment_size):
    # Calculate the number of segments
    n_segments = len(signal_profile) // segment_size
    segments = signal_profile[:n_segments * segment_size].reshape(n_segments, segment_size)

    # Lists to store the Hurst exponents and fluctuations
    Hurst_exponent = []
    fractal_dimensions = []
    
    # Iterate over each segment to calculate the fluctuation function
    for i in range(n_segments):
        segment_data = segments[i]
        
        # List to store fluctuation function for multiple segment sizes
        fluctuation_function = []
        
        # Define the range of segment sizes to compute the fluctuation function
        min_segment_size = 6 # A reasonable lower bound for segment size
        max_segment_size = len(segment_data)
        segment_sizes = 2 ** np.arange(int(np.log2(min_segment_size)), int(np.log2(max_segment_size)) + 1)
        
        for segment_size in segment_sizes:
            segment_subsets = segment_data[:segment_size * (len(segment_data) // segment_size)]
            reshaped_segments = segment_subsets.reshape(len(segment_subsets) // segment_size, segment_size)
            detrended_segments = np.apply_along_axis(detrend_segment, 1, reshaped_segments)
            segment_rms = np.sqrt(np.mean(detrended_segments ** 2, axis=1))
            fluctuation_function.append(np.mean(segment_rms) / len(segment_subsets))

        m, b = np.polyfit(np.log(segment_sizes), np.log(fluctuation_function), 1)
        segment_alpha = m / 2
        Hurst_exponent.append(segment_alpha)

        fractal_dimension = 2 - segment_alpha
        fractal_dimensions.append(fractal_dimension)

    return np.array(Hurst_exponent), np.array(fractal_dimensions)